In [1]:
import pandas as pd 
import numpy as np
import timeit


In [2]:
def app_roll_mean(df, window):
    roll  = df.rolling(window).mean() 
    roll.dropna(inplace=True)
    roll = roll.round(decimals=0).copy()
    roll.reset_index(drop=True, inplace=True)
    return roll.copy()

In [3]:
#guess window_lag = ceil(delay_b_c/t_sample)

def interrelation(window_lag, matrix):
    #window_lag =  5#shoud be minor than size of the series
    count_occor = 0;
    corr_mat = np.zeros([matrix.shape[1],matrix.shape[1]], dtype=float)
    alrms = {}
    for i in np.arange(0, matrix.shape[1]):
        alrms[i] = matrix[:,i]
    #print(alrms)
    for idx_caus, value in alrms.items():
        reduced_alarms = alrms.copy()
        del reduced_alarms[idx_caus]
        for idx_occor_caus, occor in enumerate(value):
            if occor == 1:
                if(idx_occor_caus+window_lag < len(value)):
                    count_occor = count_occor + 1
                    for idx_fct, alm_redu in reduced_alarms.items():
                        counted = False
                        for idx_occor_fct in np.arange(idx_occor_caus+1, idx_occor_caus+1+window_lag):
                            if alm_redu[idx_occor_fct] == 1 and  not counted:
                                counted = True
                                corr_mat[idx_caus][idx_fct] = corr_mat[idx_caus][idx_fct]+1
        if count_occor > 0: 
            corr_mat[idx_caus] = corr_mat[idx_caus]/count_occor
            count_occor = 0
    return corr_mat

In [4]:
cen_normal = "normal/inter/"
cen_media2 = "mean2/inter/"
cen_media5 = "mean5/inter/"

In [5]:
dist6_novo = pd.read_csv("../dist6_3horas_sig/alm_seq.csv")
dist = dist6_novo[['xmeas%02d_low' % x for x in [1,2,3,8,9,21]] + ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]]

In [6]:
df_mean2 = app_roll_mean(dist, 2)
df_mean5 = app_roll_mean(dist, 5)
w = 33 #~- 20 minutes

In [13]:
%timeit -r100 interrelation(w, dist.as_matrix())

1.91 s ± 35.3 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [25]:
time_exec_series =  pd.Series(timeit.timeit(lambda : interrelation(w, dist.as_matrix()), number =100))
df_normal_time = pd.DataFrame(time_exec_series, columns=['Tempo (s)'])
df_normal_time.to_csv(cen_normal+"df_normal_time.csv")

In [12]:
%timeit -r100 interrelation(w, df_mean2.as_matrix())

1.54 s ± 22.9 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [26]:
time_exec_series =  pd.Series(timeit.timeit(lambda : interrelation(w, df_mean2.as_matrix()), number= 100))
df_mean2_time = pd.DataFrame(time_exec_series, columns=['Tempo (s)'])
df_mean2_time.to_csv(cen_media2+"df_media2_time.csv")

In [11]:
%timeit -r100 interrelation(w, df_mean5.as_matrix())

1.67 s ± 21 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [27]:
time_exec_series =  pd.Series(timeit.timeit(lambda : interrelation(w, df_mean5.as_matrix()), number=100))
df_mean5_time = pd.DataFrame(time_exec_series, columns=['Tempo (s)'])
df_mean5_time.to_csv(cen_media5+"df_media5_time.csv")